In [3]:
from openai import OpenAI
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel
import re, json, os
from typing import Optional, Type, TypeVar, List, Dict, Any
import llm_qa_data_builder
from datetime import datetime
# import pandas as pd

In [4]:
# %pip install openai

Test chat and prompt

In [5]:
!which python

/home/ubuntu/anaconda3/bin/python


In [6]:
def load_azure_config(config_path="azure_config.json"):
    """Load Azure OpenAI configuration from JSON file."""
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        print(f"Configuration file {config_path} not found. Please create it with your Azure OpenAI settings.")
        return None
    except json.JSONDecodeError:
        print(f"Error parsing {config_path}. Please check the JSON format.")
        return None

# Load configuration
config = load_azure_config()
if not config:
    raise Exception("Failed to load Azure configuration")

In [7]:
import os
from openai import AzureOpenAI

model = "gpt-4o-mini-chat"
endpoint = config['azure_openai']['endpoint']
model_name = config['models'][model]['model_name']
deployment = config['models'][model]['deployment_name']

subscription_key = config['azure_openai']['api_key']
api_version = config['azure_openai']['api_version']

print(f"API Key loaded: {subscription_key is not None}")

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=0.7,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

API Key loaded: True
Paris is a city rich in history, culture, and beauty. Here are some must-see attractions and experiences:

1. **Eiffel Tower**: No trip to Paris is complete without a visit to this iconic landmark. You can take an elevator to the top for stunning views of the city.

2. **Louvre Museum**: Home to thousands of works of art, including the Mona Lisa and the Venus de Milo. It's a vast museum, so consider spending a few hours or even a day here.

3. **Notre-Dame Cathedral**: Although it suffered a fire in 2019, the exterior and the surrounding area are still worth visiting. You can admire its Gothic architecture and explore Île de la Cité.

4. **Montmartre and Sacré-Cœur**: Wander through the charming streets of Montmartre, visit the Basilica of Sacré-Cœur, and enjoy panoramic views of the city.

5. **Champs-Élysées and Arc de Triomphe**: Stroll down this famous avenue lined with shops and cafes, and visit the monumental Arc de Triomphe.

6. **Seine River Cruise**: Consi

In [103]:
# # Modify OpenAI's API key and API base to use vLLM's API server.
# openai_api_key = "EMPTY"
# openai_api_base = "http://host.docker.internal:8000/v1"
# client = OpenAI(
#     api_key=openai_api_key,
#     base_url=openai_api_base,
# )
# model = 'Qwen/QwQ-32B-AWQ'

In [104]:
# # Round 1
# messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
# # For granite, add: `extra_body={"chat_template_kwargs": {"thinking": True}}`
# response = client.chat.completions.create(model=model, messages=messages)

# reasoning_content = response.choices[0].message.reasoning_content
# content = response.choices[0].message.content

# print("reasoning_content:", reasoning_content)
# print("content:", content)

In [38]:
response = client.chat.completions.create(
    messages=[{"role": "system", "content": "You are a medical assistant with expertise in understanding clinical timelines and temporal relationships between medical events in clinical text."}, {"role": "user", "content": "\nGiven the clinical text, identify the precise date or time when events marked with <E#> and </E#> occurred.\n\nIMPORTANT FORMATTING INSTRUCTIONS:\n\n    - For specific dates: Use \"ON YYYY-MM-DD\" (e.g., \"ON 2023-05-15\")\n\n    - For approximate dates: Use \"IN YYYY\" for years or \"IN YYYY-MM\" for months (e.g., \"IN 1991\", \"IN 2023-05\")\n\n    - For relative timings: Use \"BEFORE YYYY-MM-DD\" or \"AFTER YYYY-MM-DD\"\n\n    - For ranges: Use \"YYYY-MM-DD TO YYYY-MM-DD\"\n\n    - If there is no temporal information: Use \"None\"\n\n    - Always select the most precise format possible, but do not guess or over-specify if the text does not support it.\n\nKEY PRINCIPLES: \n- For relative references (such as \"yesterday\", \"last week\", \"last winter\"), interpret them using a reference date.\n- For approximate dates (such as \"around May\"), use the most specific form you can justify.\n- If several dates are possible, choose the one most relevant to the clinical context.\n\nTIMESTAMP TAGS: Look for <T0>, <T1>, etc. as temporal references (e.g., <T0>September 15, 2023</T0>).\nReturn your answer in JSON format, like this:\n\n{\n  \"datetime\": \"[formatted date/time]\",\n  \"clues\": \"[specific text evidence supporting your decision]\"\n}\n\nExamples:\n\n    {\"datetime\": \"ON 2023-05-15\", \"clues\": \"procedure performed on May 15, 2023\"}\n\n    {\"datetime\": \"IN 2022\", \"clues\": \"The patient was admitted in 2024, with a diabetes diagnosis two years ago\"}\n\n    {\"datetime\": \"IN 1992-10\", \"clues\": \"CT scan performed in October 1992\"}\n\n    {\"datetime\": \"BEFORE 2023-05-15\", \"clues\": \"symptoms started prior to May 15, 2023\"}\n\n    {\"datetime\": \"2023-05-15 TO 2023-05-20\", \"clues\": \"treatment from May 15 through May 20, 2023\"}\n\n    {\"datetime\": \"None\", \"clues\": \"no temporal information found\"}\n    \n\n\nCLINICAL TEXT: Admission Date :\n09/29/1993\nDischarge Date :\n10/04/1993\nHISTORY OF PRESENT ILLNESS :\nThe patient is a 28-year-old woman who is <E2>HIV positive</E2> for two years .\nShe presented with <E4>left upper quadrant pain</E4> as well as <E5>nausea</E5> and <E6>vomiting</E6> which is <E7>a long-standing complaint</E7> .\nShe was diagnosed in 1991 during the birth of her child .\nShe claims she does not know why she is <E10>HIV positive</E10> .\nShe is from Maryland , apparently had no <E11>blood transfusions</E11> before the birth of her children so it is presumed <E13>heterosexual transmission</E13> .\nAt that time , she also had <E14>cat scratch fever</E14> and she had <E15>resection</E15> of <E16>an abscess in the left lower extremity</E16> .\nShe has not used <E17>any anti retroviral therapy</E17> since then , because of <E18>pancytopenia</E18> and <E19>vomiting</E19> on <E20>DDI</E20> .\nShe has complaints of <E21>nausea</E21> and <E22>vomiting</E22> as well as <E23>left upper quadrant pain</E23> on and off getting progressively worse over the past month .\nShe has had <E25>similar pain</E25> intermittently for last year .\nShe described <E26>the pain</E26> as <E27>a burning pain</E27> which is positional , worse when she walks or does any type of exercise .\nShe has no relief from <E32>antacids</E32> or <E33>H2 blockers</E33> .\nIn 10/92 , she had <E34>a CT scan</E34> which showed <E35>fatty infiltration of her liver diffusely</E35> with <E36>a 1 cm cyst in the right lobe of the liver</E36> .\nShe had a normal pancreas at that time , however , <E38>hyperdense kidneys</E38> .\n<E39>Her alkaline phosphatase</E39> was <E40>slightly elevated</E40> but otherwise relatively normal .\n<E42>Her amylase</E42> was <E43>mildly elevated</E43> but has been down since then .\nThe patient has had <E45>progressive failure to thrive</E45> and <E46>steady weight loss</E46> .\nShe was brought in for <E47>an esophagogastroduodenoscopy</E47> on 9/26 but she basically was not sufficiently <E48>sedated</E48> and readmitted at this time for <E50>a GI work-up</E50> as well as <E51>an evaluation</E51> of <E52>new abscess in her left lower calf</E52> and right medial lower extremity quadriceps muscle .\nShe was also admitted to be connected up with social services for <E55>HIV</E55> patients .\nHOSPITAL COURSE :\nThe patient was admitted and <E57>many cultures</E57> were sent which were all negative .\nShe did not have any of <E59>her pain</E59> in the hospital .\nOn the third hospital day , she did have <E61>some pain</E61> and was treated with <E62>Percocet</E62> .\nShe went for <E63>a debridement</E63> of <E64>her left calf lesion</E64> on 10/2/93 and was started empirically on <E65>IV ceftriaxone</E65> which was changed to <E66>po doxycycline</E66> on the day of discharge .\n<E68>A follow-up CT scan</E68> was done which did not show any <E70>evidence</E70> for <E71>splenomegaly</E71> or <E72>hepatomegaly</E72> .\n<E73>The 1 cm cyst</E73> which was seen in 10/92 was still present .\nThere was a question of <E74>a cyst in her kidney</E74> with <E75>a stone</E75> right below <E76>the cyst</E76> , although this did not seem to be clinically significant ."}],
    temperature=0.2,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

```json
{
  "events": [
    {
      "datetime": "ON 1991-00-00",
      "clues": "She was diagnosed in 1991 during the birth of her child."
    },
    {
      "datetime": "ON 1992-10-00",
      "clues": "In 10/92, she had a CT scan which showed fatty infiltration of her liver."
    },
    {
      "datetime": "ON 1993-09-26",
      "clues": "She was brought in for an esophagogastroduodenoscopy on 9/26."
    },
    {
      "datetime": "ON 1993-10-02",
      "clues": "She went for a debridement of her left calf lesion on 10/2/93."
    },
    {
      "datetime": "ON 1993-09-29",
      "clues": "Admission Date: 09/29/1993."
    },
    {
      "datetime": "ON 1993-10-04",
      "clues": "Discharge Date: 10/04/1993."
    },
    {
      "datetime": "None",
      "clues": "No specific date for the patient's long-standing complaints of nausea and vomiting."
    },
    {
      "datetime": "None",
      "clues": "No specific date for the patient's progressive failure to thrive and steady weight los

## Error analysis

In [6]:
def get_gold_standard():
    path = '/home/ubuntu/work/Temporal_relation/'
    data_dir = path + "data/timeline_training/"
    results = {}

    data = llm_qa_data_builder.data_load(data_dir, True)

    for key in data:
        ground_truth = {}
        text = data[key]['label']
        event_start_time = data[key]['starttime']
        for event in event_start_time:
            ground_truth[event['node_id']] = {
                "event_id": event['node_id'],
                "formatted_time_range": event['formatted_time_range'],
                'original_event': event
            }
        # print(ground_truth)
        results[key] = ground_truth
    return results

In [7]:
def parse_jsonl_extract_content(jsonl_file_path):
    """
    Parse a JSONL file and extract message.content from Azure OpenAI responses
    Handle partially valid JSON by extracting valid objects and skipping invalid parts
    
    Args:
        jsonl_file_path (str): Path to the JSONL file
        
    Returns:
        dict: Dictionary with custom_id as key and message content as value
    """
    results = {}
    
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
                
            try:
                # Parse the JSON line
                data = json.loads(line)
                
                # Extract custom_id
                custom_id = data.get('custom_id')
                
                
                # Navigate to message.content
                content = (data.get('response', {})
                          .get('body', {})
                          .get('choices', [{}])[0]
                          .get('message', {})
                          .get('content', None))
                
                if content:
                    if '```json' in content:
                        # Extract JSON from markdown code block
                        json_start = content.find('```json') + 7
                        json_end = content.find('```', json_start)
                        if json_end != -1:
                            json_content = content[json_start:json_end].strip()
                        else:
                            # No closing ```, try to parse from json_start to end
                            json_content = content[json_start:].strip()
                    else:
                        json_content = content.strip()
                    
                    # Parse partially valid JSON
                    parsed_content = parse_partial_json(json_content)
                    
                    if custom_id and parsed_content:
                        results[custom_id] = parsed_content
                    # if custom_id == 'doc-491-task-E9':
                    #     print(f"Custom ID: {custom_id}")
                    #     print(f"Parsed Content: {json_content} \n {parsed_content}")    
            except (json.JSONDecodeError, IndexError, KeyError) as e:
                print(f"Error parsing line: {e}")
                print(f"Line content: {line}")
                continue
    
    return results

In [8]:
def parse_partial_json(json_string):
    """
    Parse partially valid JSON by extracting complete JSON objects
    
    Args:
        json_string (str): JSON string that may be partially valid
        
    Returns:
        dict or list: Single dict if one object found, list if multiple objects found, None if no valid objects
    """
    valid_objects = []
    
    # First try to parse as complete JSON
    try:
        result = json.loads(json_string)
        # If it's a list, filter out empty dictionaries
        if isinstance(result, list):
            non_empty_objects = [obj for obj in result if obj and obj != {}]
            if len(non_empty_objects) == 1:
                return non_empty_objects[0]
            elif len(non_empty_objects) > 1:
                return non_empty_objects
            else:
                return None
        return result
    except json.JSONDecodeError:
        pass
    
    # If that fails, try to extract individual objects
    # Look for patterns like { ... } that might be valid JSON objects
    brace_count = 0
    start_pos = -1
    i = 0
    
    while i < len(json_string):
        char = json_string[i]
        
        if char == '{':
            if brace_count == 0:
                start_pos = i
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0 and start_pos != -1:
                # Found a complete object
                potential_object = json_string[start_pos:i+1]
                try:
                    parsed_obj = json.loads(potential_object)
                    # Only add non-empty objects
                    if parsed_obj and parsed_obj != {}:
                        valid_objects.append(parsed_obj)
                except json.JSONDecodeError:
                    # Skip invalid object
                    pass
                start_pos = -1
        
        i += 1
    
    # Return single object if only one found, list if multiple, None if none
    if len(valid_objects) == 1:
        return valid_objects[0]
    elif len(valid_objects) > 1:
        return valid_objects
    else:
        return None

In [9]:
def get_azure_annotation_all(jsonl_file_path):
    """
    Load Azure OpenAI annotations from JSONL file
    """
    
    # Parse the JSONL and extract content
    annotations = parse_jsonl_extract_content(jsonl_file_path)
    
    formatted_annotations = {}
    for cid in annotations.keys():
        tokens = cid.split('-')
        doc_id = tokens[1]
        if doc_id not in formatted_annotations:
            formatted_annotations[doc_id] = {}
            for event in annotations[cid]:
                if isinstance(event, dict) and 'event_id' in event:
                    formatted_annotations[doc_id][event['event_id']] = event
                else:
                    print(f"Missing event_id in {cid}: {event}")

    return formatted_annotations

In [10]:
def get_azure_annotation_individual(jsonl_file_path):
    """
    Load Azure OpenAI annotations from JSONL file for individual task format
    """
    # Update this path to your actual JSONL file location
       # Parse the JSONL and extract content
    annotations = parse_jsonl_extract_content(jsonl_file_path)
    # print(annotations)
    
    formatted_annotations = {}
    for key in annotations.keys():
        tokens = key.split('-')
        doc_id, event_id = tokens[1], tokens[-1]
        if doc_id not in formatted_annotations:
            formatted_annotations[doc_id] = {}
        formatted_annotations[doc_id][event_id] = annotations[key]

    return formatted_annotations

In [11]:
def parse_datetime(date_str):
    # print(date_str)
    """Parse a date string into a datetime object, handling various formats."""
    import re
    
    # Handle various date formats and normalize them
    for old, new in [('DURING ', ''), ('BETWEEN ', ''), ('FROM ', ''), ('MIDWAY THROUGH ', ''), ('-??', '-01'), ('-XX', '-01'), ('-xx', '-01'), ('-00', '-01')]:
        date_str = date_str.replace(old, new)
        
    # Remove time component if present
    if "T" in date_str:
        date_str = date_str.split('T')[0]
    
    # Remove time component with space separator - updated pattern to handle a.m./p.m.
    time_patterns = [
        r'\s+\d{1,2}:\d{2}(:\d{2})?\s*[ap]\.?m\.?',  # Handles "2:30 p.m.", "2:30 pm", "2:30p.m."
        r'\s+\d{1,2}:\d{2}(:\d{2})?'                 # Handles regular "14:30", "2:30"
    ]
    
    for pattern in time_patterns:
        date_str = re.sub(pattern, '', date_str, flags=re.IGNORECASE)

    # Handle "Month DD, YYYY" format first (like "February 18, 1994")
    month_day_year_pattern = r'(\w+)\s+(\d+),?\s+(\d{4})'
    match = re.match(month_day_year_pattern, date_str)
    if match:
        month_name, day, year = match.groups()
        try:
            # Try full month name first
            date = datetime.strptime(f"{month_name} {day} {year}", '%B %d %Y')
            return date
        except ValueError:
            try:
                # Try abbreviated month name
                date = datetime.strptime(f"{month_name} {day} {year}", '%b %d %Y')
                return date
            except ValueError:
                pass
    
    # Handle "Month DD TO Month DD, YYYY" format (like "MARCH 14 TO MARCH 21, 1994")
    month_range_pattern = r'(\w+)\s+(\d+)\s+TO\s+(\w+)\s+(\d+),?\s+(\d{4})'
    match = re.match(month_range_pattern, date_str)
    if match:
        month1, day1, month2, day2, year = match.groups()
        try:
            # Return the start date for range formats
            date = datetime.strptime(f"{month1} {day1} {year}", '%B %d %Y')
            return date
        except ValueError:
            try:
                date = datetime.strptime(f"{month1.title()} {day1} {year}", '%B %d %Y')
                return date
            except ValueError:
                pass
    
    # Handle month name + year format
    if any(month in date_str.upper() for month in ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE',
                                          'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER']):
        try:
            # Handle "MONTH YYYY" format
            if any(month in date_str for month in ['January', 'February', 'March', 'April', 'May', 'June',
                                                  'July', 'August', 'September', 'October', 'November', 'December']):
                date = datetime.strptime(date_str, '%B %Y')
                return date
            # Handle "MONTH YYYY" format (uppercase)
            elif any(month in date_str for month in ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE',
                                                    'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER']):
                date = datetime.strptime(date_str.title(), '%B %Y')
                return date
        except ValueError:
            # If that fails, try abbreviated month names
            try:
                date = datetime.strptime(date_str, '%b %Y')
                return date
            except ValueError:
                pass
    
    # Handle YYYY-MM-DD TO YYYY-MM-DD format
    if 'TO' in date_str and re.search(r'\d{4}-\d{1,2}-\d{1,2}', date_str):
        date_parts = date_str.split('TO')
        if len(date_parts) == 2:
            start_date_str = date_parts[0].strip()
            # Use the start date
            date_str = start_date_str
    
    # Handle prefixes like "NIGHT BEFORE", "ON", "BEFORE", etc.
    if ' ' in date_str:
        # Look for date patterns in the string
        date_pattern = r'\b\d{4}-\d{1,2}-\d{1,2}\b'
        date_match = re.search(date_pattern, date_str)
        if date_match:
            date_str = date_match.group()
        else:
            # Look for year patterns
            year_pattern = r'\b(19|20)\d{2}\b'
            year_match = re.search(year_pattern, date_str)
            if year_match:
                date_str = year_match.group()
            else:
                # Fall back to last word if no clear date pattern
                words = date_str.split(' ')
                date_str = words[-1]
    
    if ',' in date_str:
        date_str = date_str.split(',')[0]

    # Handle MM-YYYY format (like "12-1992")
    if re.match(r'^\d{1,2}-\d{4}$', date_str):
        month, year = date_str.split('-')
        date_str = f"{year}-{month.zfill(2)}-01"

    # Handle MM/DD/YYYY and M/D/YY formats (like 08/15/1998 or 9/7/93)
    if '/' in date_str:
        parts = date_str.split('/')
        if len(parts) == 3:
            month, day, year = parts
            # Handle 2-digit years
            if len(year) == 2:
                year_int = int(year)
                # Assume years 00-30 are 2000s, 31-99 are 1900s
                year = f"20{year}" if year_int <= 30 else f"19{year}"
            # Convert to YYYY-MM-DD format with zero padding
            date_str = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
        elif len(parts) == 2:
            # Handle MM/YY format (like "10/92")
            month, year = parts
            if len(year) == 2:
                year_int = int(year)
                year = f"20{year}" if year_int <= 30 else f"19{year}"
            date_str = f"{year}-{month.zfill(2)}-01"
    
    # Handle dash-separated dates (MM-DD-YYYY, MM-DD-YY, YYYY-MM-DD)
    if '-' in date_str:
        parts = date_str.split('-')
        if len(parts) == 3:
            # Check if it's MM-DD-YYYY format (last part is 4 digits)
            if len(parts[2]) == 4:  # MM-DD-YYYY or M-DD-YYYY or MM-D-YYYY
                month, day, year = parts
                date_str = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
            # Check if it's MM-DD-YY format (last part is 2 digits and first two parts are reasonable dates)
            elif (len(parts[2]) == 2 and 
                  int(parts[0]) <= 12 and int(parts[1]) <= 31):  # M-DD-YY or MM-D-YY or MM-DD-YY
                month, day, year = parts
                year_int = int(year)
                year_prefix = '19' if year_int >= 50 else '20'
                date_str = f"{year_prefix}{year}-{month.zfill(2)}-{day.zfill(2)}"
            # Check if it's already YYYY-MM-DD format (first part is 4 digits)
            elif len(parts[0]) == 4:
                # Already in YYYY-MM-DD format, just ensure zero padding
                year, month, day = parts
                date_str = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
        elif len(parts) == 2:
            # Check if it's MM-DD format (both parts are reasonable dates)
            if (int(parts[0]) <= 12 and int(parts[1]) <= 31):
                # MM-DD format - assume current year
                current_year = datetime.now().year
                date_str = f"{current_year}-{parts[0].zfill(2)}-{parts[1].zfill(2)}"
            else:
                # Format is YYYY-MM (already handled above for MM-YYYY)
                date_str = f"{date_str}-01"
    
    # Handle year-only format
    if '-' not in date_str and re.match(r'^\d{4}$', date_str):
        # Year only
        date_str = f"{date_str}-01-01"
    
    # Final parsing
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        return date
    except ValueError as e:
        # If still can't parse, try to extract just the year
        year_match = re.search(r'\b(19|20)\d{2}\b', date_str)
        if year_match:
            year = year_match.group()
            return datetime.strptime(f"{year}-01-01", '%Y-%m-%d')
        raise e

In [12]:
def parse_label(label):
    # print(f"Parsing label: {label}")
    if (label is None) or (label == 'None') or ('None' in label):
        return None
    
    # Handle "UNKNOWN DATE" case
    if 'UNKNOWN DATE' in label.upper():
        return None
    
    # Check if label contains any date-like information
    # Look for years (4 digits), months (1-12), or common date patterns
    import re
    
    # Pattern to match years (1800-2099), months (01-12 or 1-12), days (01-31 or 1-31)
    # Also check for month names and common date separators
    date_patterns = [
        r'\b(18|19|20)\d{2}\b',  # Years 1800-2099
        r'\b(0?[1-9]|1[0-2])[/-](0?[1-9]|[12][0-9]|3[01])[/-](\d{2}|\d{4})\b',  # MM/DD/YY or MM/DD/YYYY
        r'\b(0?[1-9]|[12][0-9]|3[01])[/-](0?[1-9]|1[0-2])[/-](\d{2}|\d{4})\b',  # DD/MM/YY or DD/MM/YYYY
        r'\b(January|February|March|April|May|June|July|August|September|October|November|December)\b',  # Full month names
        r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b',  # Abbreviated month names
        r'\b(0?[1-9]|1[0-2])[/-](0?[1-9]|[12][0-9]|3[01])\b',  # MM/DD or MM-DD
    ]
    
    has_date_info = any(re.search(pattern, label, re.IGNORECASE) for pattern in date_patterns)
    
    if not has_date_info:
        # print(f"No date information found in label: {label}")
        return None
    
    if label.startswith('AFTER ON'):
        label = label.replace('AFTER ON', 'AFTER')
    elif label.startswith('AFTER OR ON'):
        label = label.replace('AFTER OR ON', 'AFTER')
    elif label.startswith('BEFORE ON'):
        label = label.replace('BEFORE ON', 'BEFORE')
    elif label.startswith('BEFORE OR ON'):
        label = label.replace('BEFORE OR ON', 'BEFORE')
    elif label.startswith('AT'):
        label = label.replace('AT', 'ON')

    try:
        if label.startswith('ON'):
            # Handle cases like "ON NIGHT BEFORE 2019-06-14" or "ON February 18, 1994"
            remaining = ' '.join(label.split()[1:])  # Everything after "ON"
            date = parse_datetime(remaining)
            return 'ON', date, date
        elif label.startswith('BEFORE'):
            # Handle cases like "BEFORE February 18, 1994"
            remaining = ' '.join(label.split()[1:])  # Everything after "BEFORE"
            date = parse_datetime(remaining)
            return 'BEFORE', datetime.min, date
        elif label.startswith('AFTER'):
            # Handle cases like "AFTER February 18, 1994"
            remaining = ' '.join(label.split()[1:])  # Everything after "AFTER"
            date = parse_datetime(remaining)
            return 'AFTER', date, datetime.max
        elif 'TO' in label:
            # Split more carefully for TO cases
            if 'TO UNKNOWN DATE' in label.upper():
                # Handle cases like "10/92 TO UNKNOWN DATE"
                start_part = label.split('TO')[0].strip()
                d1 = parse_datetime(start_part)
                return 'TO', d1, datetime.max
            else:
                # Handle cases like "MARCH 14 TO MARCH 21, 1994"
                parts = label.split('TO')
                if len(parts) == 2:
                    start_part = parts[0].strip()
                    end_part = parts[1].strip()
                    d1 = parse_datetime(start_part)
                    d2 = parse_datetime(end_part)
                    return 'TO', d1, d2
    except (ValueError, IndexError) as e:
        print(f"Error parsing date from label '{label}': {e}")
        return None
    
    return None


In [13]:
# Function to check interval overlap
def intervals_overlap(gt_interval, pred_interval):
    if not gt_interval or not pred_interval:
        return False
    label_gt, start_gt, end_gt = gt_interval
    label_pred, start_pred, end_pred = pred_interval
    return (label_gt==label_pred) & (max(start_gt, start_pred) <= min(end_gt, end_pred))



In [14]:
def evaluate_azure_annotations(annotations, gold_standard):
    """
    Evaluate Azure OpenAI annotations against the gold standard
    """
    # Evaluate
    # Initialize counters for each category
    categories = ["ON", "BEFORE", "AFTER", "TO"]
    category_totals = {cat: 0 for cat in categories}
    category_strict_matches = {cat: 0 for cat in categories}
    category_relaxed_matches = {cat: 0 for cat in categories}
    # Overall counters
    total = 0
    strict_match = 0
    relaxed_match = 0

    for record_id, entries in gold_standard.items():
        total += len(entries)
        for event_id in entries.keys():
            truth = entries[event_id]['formatted_time_range']
            gt = parse_label(truth)
            prediction = annotations.get(record_id, {}).get(event_id, {}).get('datetime', None)
            clues = annotations.get(record_id, {}).get(event_id, {}).get('clues', None)
            pred = parse_label(prediction)
            event = entries[event_id]['original_event']
            
            if gt is None:
                continue
                
            category = gt[0]
            category_totals[category] += 1

            is_strict_match = gt == pred
            is_relaxed_match = is_strict_match or (intervals_overlap(gt, pred))
            if is_strict_match:
                strict_match += 1
                category_strict_matches[category] += 1
            else:
                print('-'*20)
                print("Original event:\n", json.dumps(event, indent=2, ensure_ascii=False))
                print("Truth: ",truth)
                print("Prediction: ", prediction)
                print("Parsed truth: ", gt)
                print("Parsed prediction: ", pred)
                print("Clues: ", clues)
            if is_relaxed_match:
                relaxed_match += 1
                category_relaxed_matches[category] += 1

    # Calculate overall accuracy
    strict_accuracy = strict_match / total if total > 0 else 0
    relaxed_accuracy = relaxed_match / total if total > 0 else 0

    # Print results without pandas
    print("Overall Results:")
    print(f"Total Samples: {total}")
    print(f"Strict Matches: {strict_match}")
    print(f"Strict Accuracy: {strict_accuracy:.4f}")
    print(f"Relaxed Matches: {relaxed_match}")
    print(f"Relaxed Accuracy: {relaxed_accuracy:.4f}")
    
    print("\nCategory Breakdown:")
    print(f"{'Category':<10} {'Total':<8} {'Strict Matches':<15} {'Strict Acc':<12} {'Relaxed Matches':<16} {'Relaxed Acc':<12}")
    print("-" * 80)
    
    for cat in categories:
        cat_total = category_totals[cat]
        strict_acc = category_strict_matches[cat] / cat_total if cat_total > 0 else 0
        relaxed_acc = category_relaxed_matches[cat] / cat_total if cat_total > 0 else 0
        print(f"{cat:<10} {cat_total:<8} {category_strict_matches[cat]:<15} {strict_acc:<12.4f} {category_relaxed_matches[cat]:<16} {relaxed_acc:<12.4f}")


In [15]:
gold_standard = get_gold_standard()
print(f"Gold standard loaded with {len(gold_standard)} records.")

Loaded data for 190 IDs.
Gold standard loaded with 190 records.


In [17]:
for i in range(1, 2):
    path = "/home/ubuntu/work/Temporal_relation/llm_qa/GPT4/" + 'file' + str(i) + '/'
    directory = os.fsencode(path)

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if 'results' in filename:
            if "all" in filename: 
                # print(os.path.join(directory, filename))
                annotations = get_azure_annotation_all(path+filename)
            else:
                annotations = get_azure_annotation_individual(path+filename)
            print(filename)
            evaluate_azure_annotations(annotations, gold_standard)

timeline_azure_bulk_openai_time_individual_results.jsonl
--------------------
Original event:
 {
  "node_id": "E10",
  "type": "PROBLEM",
  "text": "HIV positive",
  "start_range": [
    "1991-09-29 00:00:00",
    "1991-09-29 23:59:59.999999"
  ],
  "formatted_time_range": "ON 1991-09-29",
  "context_time_range": "09/29/1993 - two years TO 09/29/1993 - two years",
  "paths": [
    "(E10: HIV positive) -> [SIMULTANEOUS] -> (E2: HIV positive) -> [SIMULTANEOUS] -> (T2: two years) -> [ENDED_BY] -> (T0: 09/29/1993)"
  ]
}
Truth:  ON 1991-09-29
Prediction:  1991
Parsed truth:  ('ON', datetime.datetime(1991, 9, 29, 0, 0), datetime.datetime(1991, 9, 29, 0, 0))
Parsed prediction:  None
Clues:  The text states that the patient was diagnosed as HIV positive during the birth of her child in 1991, which directly corresponds to <E10>HIV positive</E10>.
--------------------
Original event:
 {
  "node_id": "E2",
  "type": "PROBLEM",
  "text": "HIV positive",
  "start_range": [
    "1991-09-29 00:00:00

KeyboardInterrupt: 

## Build prompts

In [1]:
import json

In [3]:
# common = """
# Given the clinical text, identify the precise date or time when events marked with <E#> and </E#> occurred.

# IMPORTANT FORMATTING INSTRUCTIONS:

#     - For specific dates: Use "ON YYYY-MM-DD" (e.g., "ON 2023-05-15")

#     - For approximate dates: Use "IN YYYY" for years or "IN YYYY-MM" for months (e.g., "IN 1991", "IN 2023-05")

#     - For relative timings: Use "BEFORE YYYY-MM-DD" or "AFTER YYYY-MM-DD"

#     - For ranges: Use "YYYY-MM-DD TO YYYY-MM-DD"

#     - If there is no temporal information: Use "None"

#     - Always select the most precise format possible, but do not guess or over-specify if the text does not support it.

# KEY PRINCIPLES: 
# - For relative references (such as "yesterday", "last week", "last winter"), interpret them using a reference date.
# - For approximate dates (such as "around May"), use the most specific form you can justify.
# - If several dates are possible, choose the one most relevant to the clinical context.

# """



In [ ]:
common = """

Given the clinical text, determine the exact start date or time for events enclosed by <E#> and </E#> tags.

The identified start time must strictly follow one of the six allowed formats — no deviations are permitted:

* AT <timestamp> – For specific moments in time (e.g., AT 2002-02-01 14:00:00)
* ON <date> – For precise calendar dates (e.g., ON 2002-02-01)
* IN <month> or IN <year> – For approximate periods (e.g., IN 2002-02, IN 2002)
* BEFORE <timestamp> – If the event occurred before a known time
* AFTER <timestamp> – If the event occurred after a known time
* <timestamp1> TO <timestamp2> – For events spanning a known interval

All timestamps must be in one of the following formats: YYYY-MM-DD HH:MM:SS, YYYY-MM-DD, YYYY-MM, or YYYY.

---

Special Instructions and Edge Cases:

1. **Vague or Approximate Expressions**  
   If the time is inferred from imprecise language (e.g., “several years ago”), provide a best estimate and prefix it with `APPROXIMATELY`. Examples:
   * APPROXIMATELY IN 2001  
   * APPROXIMATELY 1999-01-01 TO 2000-01-01

2. **Negative Events**  
   For events described as *not* occurring, estimate when the absence began — not when it was recorded. Use contextual clues.  
   For example, if a note says, “the patient has been coughing for two years but never had a fever,” the negative event “fever” should be dated to when the cough began — two years ago.

3. **Planned or Future Events**  
   For events scheduled but not yet occurred (e.g., planned procedures), estimate the intended start time. This is typically based on the discharge date or any future timeframe explicitly stated in the document.

"""


In [12]:
individual_output = """
Return your answer in JSON format, like this:

{
  "datetime": "[formatted date/time]",
  "clues": "[specific text evidence supporting your decision]"
}
    
"""

all_output = """Return your answer in JSON format, like this:

[
  {
    "event_id": "[the ID of the event (e.g., "E3", "E5")]",
    "datetime": "[formatted date/time]",
    "clues": "[specific text evidence supporting your decision]"
  }
]
"""

In [13]:
time = "TIMESTAMP TAGS: Look for <T0>, <T1>, etc. as temporal references (e.g., <T0>September 15, 2023</T0>)."
section = """SECTION CONTEXT:
If the text lacks clear temporal markers, use the surrounding section context to help determine the timing of events:
- Events in 'History of Present Illness' or similar sections: use admission dates as reference points.
- Events in 'Hospital Course' or discharge sections: use discharge dates as reference points.
"""

In [14]:
keys = [
  "individual_notime",
  "individual_time",
  "individual_notime_section",  
"individual_time_section",
  "all_notime",
  "all_time",
  "all_notime_section",
  "all_time_section"
]
promots = {}


In [15]:
for k in keys:
    p = common
    if "time" in k:
        p = p + time
    if "section" in k:
        p = p + section
    if "individual" in k:
        p = p + individual_output
    else:
        p = p + all_output
    promots[k] = p

In [16]:
json.dump(promots, open("prompts.json", "w"), indent=2, ensure_ascii=False)

In [17]:
promots

{'individual_notime': '\n\nGiven the clinical text, determine the exact start date or time for events enclosed by <E#> and </E#> tags.\n\nThe identified start time must strictly follow one of the six allowed formats — no deviations are permitted:\n\n* AT <timestamp> – For specific moments in time (e.g., AT 2002-02-01 14:00:00)\n* ON <date> – For precise calendar dates (e.g., ON 2002-02-01)\n* IN <month> or IN <year> – For approximate periods (e.g., IN 2002-02, IN 2002)\n* BEFORE <timestamp> – If the event occurred before a known time\n* AFTER <timestamp> – If the event occurred after a known time\n* <timestamp1> TO <timestamp2> – For events spanning a known interval\n\nAll timestamps must follow the format: YYYY-MM-DD HH:MM:SS  \n* Include smaller units (hours, minutes, seconds) only if supported by the text  \n* Do **not** infer more specific values (e.g., month or day) unless the text clearly justifies them  \n* Use the least specific valid format (e.g., IN 1987 instead of IN 1987-11